In [3]:
### Criando um script 
import requests
import json 
import pandas as pd
from airflow import DAG

ModuleNotFoundError: No module named 'airflow'

In [97]:
### Criando variáveis
url = "https://api.openbrewerydb.org/v1/breweries"

In [98]:
# 1. Faz a requisição para a API
response = requests.get(url)

### Verificando resposta
response.status_code

200

In [105]:
# Demonstrando o retorno de dados com 2 itens por página.
param_url = {'per_page': 2,
             'page': 1 }
# Realizando um request
response = requests.request('GET', url= url, params = param_url)

response.json()

[{'id': '5128df48-79fc-4f0f-8b52-d06be54d0cec',
  'name': '(405) Brewing Co',
  'brewery_type': 'micro',
  'address_1': '1716 Topeka St',
  'address_2': None,
  'address_3': None,
  'city': 'Norman',
  'state_province': 'Oklahoma',
  'postal_code': '73069-8224',
  'country': 'United States',
  'longitude': -97.46818222,
  'latitude': 35.25738891,
  'phone': '4058160490',
  'website_url': 'http://www.405brewing.com',
  'state': 'Oklahoma',
  'street': '1716 Topeka St'},
 {'id': '9c5a66c8-cc13-416f-a5d9-0a769c87d318',
  'name': '(512) Brewing Co',
  'brewery_type': 'micro',
  'address_1': '407 Radam Ln Ste F200',
  'address_2': None,
  'address_3': None,
  'city': 'Austin',
  'state_province': 'Texas',
  'postal_code': '78745-1197',
  'country': 'United States',
  'longitude': None,
  'latitude': None,
  'phone': '5129211545',
  'website_url': 'http://www.512brewing.com',
  'state': 'Texas',
  'street': '407 Radam Ln Ste F200'}]

In [ ]:
## Abaixo a ideia é adicionar a resposta a uma lista
## permitindo eu verificar se há dados em cada chamada
## incrementando as páginas até chegar a ultima página

In [104]:
df_brew = pd.DataFrame()
n = 1

while True:
    params = {
        'per_page': 50,
        'page': n      
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Verifica erros HTTP
        
        # A API retorna uma lista diretamente, não um dicionário com 'items'
        breweries = response.json()  # Isso já é a lista de cervejarias
        
        if len(breweries) > 0:
            df_brew = pd.concat([df_brew, pd.DataFrame(breweries)])
            n += 1
        else:
            break
    except requests.exceptions.RequestException as e:
        print(f"Erro na página {n}: {e}")
        break

# Resultado final
print(f"Total de páginas coletadas: {n-1}")
print(f"Total de registros: {len(df_brew)}")
print(df_brew.head())

Total de páginas coletadas: 168
Total de registros: 8370
                                     id                     name brewery_type  \
0  5128df48-79fc-4f0f-8b52-d06be54d0cec         (405) Brewing Co        micro   
1  9c5a66c8-cc13-416f-a5d9-0a769c87d318         (512) Brewing Co        micro   
2  34e8c68b-6146-453f-a4b9-1f6cd99a5ada  1 of Us Brewing Company        micro   
3  6d14b220-8926-4521-8d19-b98a2d6ec3db     10 Barrel Brewing Co        large   
4  e2e78bd8-80ff-4a61-a65c-3bfbd9d76ce2     10 Barrel Brewing Co        large   

                     address_1 address_2 address_3            city  \
0               1716 Topeka St      None      None          Norman   
1        407 Radam Ln Ste F200      None      None          Austin   
2          8100 Washington Ave      None      None  Mount Pleasant   
3                62970 18th St      None      None            Bend   
4  1135 NW Galveston Ave Ste B      None      None            Bend   

  state_province postal_code       

In [ ]:
### Criando um script 
import requests
import json 
import pandas as pd

### Criando variável com o link do endpoint
url = "https://api.openbrewerydb.org/v1/breweries"

# Faço uma requisição
response = requests.get(url)

### Verifico resposta
##response.status_code

## Crio um dataframe
df_brew = pd.DataFrame()

n = 1

while True:
    params = {
        'per_page': 50,
        'page': n      
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Verifica erros HTTP
        
        # A API retorna uma lista diretamente, não um dicionário com 'items'
        breweries = response.json()  # Isso já é a lista de cervejarias
        
        if len(breweries) > 0:
            df_brew = pd.concat([df_brew, pd.DataFrame(breweries)])
            n += 1
        else:
            break
    except requests.exceptions.RequestException as e:
        print(f"Erro na página {n}: {e}")
        break

# Resultado final
print(f"Total de páginas coletadas: {n-1}")
print(f"Total de registros: {len(df_brew)}")
print(df_brew.head())

In [2]:
import requests
import json

### Criando variável com o link do endpoint
url = "https://api.openbrewerydb.org/v1/breweries"

# Lista para armazenar todas as cervejarias
all_breweries = []

n = 1

while True:
    params = {
        'per_page': 50,
        'page': n      
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Verifica erros HTTP
        
        breweries = response.json()  # Lista de cervejarias da página atual
        
        if len(breweries) > 0:
            all_breweries.extend(breweries)  # Adiciona à lista principal
            n += 1
        else:
            break
    except requests.exceptions.RequestException as e:
        print(f"Erro na página {n}: {e}")
        break

# Salva todos os dados em um arquivo JSON
with open('breweries.json', 'w', encoding='utf-8') as f:
    json.dump(all_breweries, f, ensure_ascii=False, indent=4)